### 一.选择模型
模型的选择也要基于业务需求来考虑，通常需要从模型的【精度、业务可解释性、系统上线难易度】等多个指标来做权衡，目前对于表格型数据，通常采用GBDT一类的模型就能取得，不错的效果，通常会用xgboost或者lightgbm，这里我们就基于lightgbm来做建模，关于它的原理，不在这里介绍，大家可以查看：https://github.com/zhulei227/ML_Notes 这个项目中的笔记

### 二.选择：损失函数 vs 评估指标

#### 2.1   什么是损失函数

损失函数，它是我们构建模型的预测$f(x)$与真实值$y$之间的一个函数($g_1$)，这个指标将会用于指导模型的训练：    

$$
loss(x,y,f)=g_1(f(x),y)
$$   

这里为了求解方便，通常要求$g_1$具有良好性质，通常至少是要可导的，对于lightgbm甚至要求$g_1$可二阶导，比如对于mse损失函数，对于$g_1$的定义：   

$$
g_1(f(x),y)=(f(x)-y)^2
$$   

而对于评估指标，它本质也是关于$f,x,y$的一个函数($g_2$)：   

$$
eval(x,y,f)=g_2(f(x),y)
$$    

而对于$g_2$就没有那么好的性质类，对于这样的评估指标："前top 5%的平均赔付率",我们需要先对$f(x)$排序，去前top 5%的$y$值求平均，这样的评估指标压根不可导，甚至没法用数学表达式，所以通常会选择一个可导的$g_1$去替代$g_2$，期望通过对$g_1$的最优化来对$g_2$进行最优化，比如下面，通过最小化mse去，最大化评估指标：测试集前top 5%的平均赔付率   

|||
|---|---|
|评估指标|测试集前top 5%的平均赔付率|
|损失函数|mse|


#### 2.2 如何选择最佳的损失函数
从上面的介绍可以知道，损失函数与我们的评估指标之间存在一个差距，那么如何找到（创造）一个很好的损失函数，使得它与评估指标之间的gap尽可能地小？这个问题很麻烦，通常来说可以尝试如下的方法：    

1）尝试最常用的一些损失函数，比如回归用mse,mae,分类用交叉熵；    

2）对有很强分布假设的数据采用对应的损失函数，比如出险次数的分布通常符合poisson分布，损失函数可以尝试使用poisson回归的损失函数；   

3）使用多种损失函数的组合，这通常需要自己去自定义实现损失函数；    

下面就用lgm训练一个模型，从mse开始

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, auc
from pre_process.xxx import *
import toad
from sklearn import metrics

In [1]:
diretory="xxx"
test0 = pd.read_csv(diretory+'xxx.csv', low_memory=False)
test0.shape

#### 3.1加工目标:出险率

In [3]:
claim = pd.read_csv(diretory+'/data/claimnum.csv')
claim = claim[['polnum_com', 'num']]
claim = claim.rename(columns={'polnum_com': 'policy_no'})
test0=pd.merge(test0,claim,on="policy_no",how="inner")
test0['y'] = np.where(test0['num']>0, 1, 0)
test0=test0.drop("num",axis=1)
#平均出险率
test0['y'].mean()

#### 3.2 数据清洗+特征工程

这里的特征工程是对全局数据进行操作，需要避免数据泄露的操作

In [5]:
dat1=test0.copy()
#因子筛选
dat1 = ppcess_filterCols(dat1, cols='join')
#去掉饱和度过低的因子 by 80%
na_cols = []
for c in dat1.columns:
    if dat1[c].isnull().sum()/len(dat1)>0.8:
        na_cols.append(c)
dat1 = dat1.drop(na_cols, axis=1)
# 去掉集团因子里面产险数据来源的因子
jt_cx_cols = ['xxx']
dat1 = dat1.drop(jt_cx_cols, axis=1)
dat1 = ppcess_dateCols(dat1,print_process_cols=False) # 日期因子格式
dat1 = ppcess_toNum(dat1,print_process_cols=False) # 部分因子转换为数值型
dat1 = ppcess_delCols_na(dat1,print_process_cols=False) # 删去饱和度低于1%的因子
dat1 = ppcess_inpute(dat1,print_process_cols=False) # 部分因子按规则进行空值填充，数值型因子在后面做均值填充
dat1 = ppcess_delCols_uniValue(dat1,print_process_cols=False) # 删去全部唯一值的因子

dat2 = dat1.copy()
dat2 = feature_catValues(dat2,print_process_cols=False) # 修正部分因子取值
dat2 = feature_factorLabel1(dat2,print_process_cols=False) # 离散特征编码1
dat2 = feature_factorLabel2(dat2,print_process_cols=False) # 离散特征编码2
dat2 = feature_lambda(dat2,print_process_cols=False) # 构建一些特征
dat2 = dat2.drop(['xxx'], axis=1)
dat2['xxx'] = dat2['xxx'].fillna(0)
dat2['y']=test0['y']

#### 3.3 切分训练集，验证集，测试集

In [6]:
trn_df=dat2[test0['effdate'].apply(lambda x:x[:7]<="2019-10")]

dev_test_df=dat2[test0['effdate'].apply(lambda x:x[:7]>="2019-11")]
indice=list(range(0,dev_test_df.shape[0]))
np.random.shuffle(indice)
dev_test_df=dev_test_df.iloc[indice]
dev_df=dev_test_df.iloc[:dev_test_df.shape[0]//2]
test_df=dev_test_df.iloc[dev_test_df.shape[0]//2:]

In [4]:
trn_df.shape,dev_df.shape,test_df.shape

In [5]:
trn_df.head()

#### 3.4 target encoding


In [9]:
object_cols=trn_df.dtypes[trn_df.dtypes==object].reset_index()['index'].tolist()
trn_df[object_cols]=trn_df[object_cols].fillna("missing")
dev_df[object_cols]=dev_df[object_cols].fillna("missing")
test_df[object_cols]=test_df[object_cols].fillna("missing")

In [10]:
object_target_cols=[]
for col in object_cols:
    object_target_cols.append(col+"_target")
    trn_df[col+"_target"]=trn_df[col]
    dev_df[col+"_target"]=dev_df[col]
    test_df[col+"_target"]=test_df[col]

In [6]:
import category_encoders as ce
le=ce.TargetEncoder(cols=object_target_cols)
le.fit(trn_df,trn_df['y'])

In [12]:
trn_df=le.transform(trn_df)
dev_df=le.transform(dev_df)
test_df=le.transform(test_df)

####  3.5 ordinary encoding

In [7]:
oe=ce.OrdinalEncoder()
oe.fit(trn_df,cols=object_cols)

In [14]:
trn_df=oe.transform(trn_df)
dev_df=oe.transform(dev_df)
test_df=oe.transform(test_df)

对于离散型变量，往往有许多编码方式，主要分为两类：    
1）一类是不与目标y交互，比如one-hot编码，ordinary encoding...    
2）另一类会与目标y交互，生成一些统计量，比如上面的target encoding,catboost...   

通常与y交互会增加当前特征的信息量，比如下面从相关性的角度去做一个对比

In [15]:
corr_df=trn_df[object_cols+object_target_cols+['y']].corr().reset_index()

In [16]:
ordinary_corr=[]
target_encode_corr=[]
for col in object_cols:
    ordinary_corr.append(corr_df[corr_df['index']==col]['y'].tolist()[0]) 
    target_encode_corr.append(corr_df[corr_df['index']==col+"_target"]['y'].tolist()[0])

In [17]:
corr_df=pd.DataFrame({"col_name":object_cols,"ordinary_corr":ordinary_corr,"target_encode_corr":target_encode_corr})
corr_df["相对提升"]=corr_df['target_encode_corr'].abs()/corr_df['ordinary_corr'].abs()-1.0

In [8]:
corr_df

### 四.模型训练

In [19]:
#自定义metrics
def eval_function(y_true,y_pred):
    try:
        y_pred=y_pred.get_label()
    except:
        pass
    sort_indice=np.argsort(y_pred)[::-1]
    metric_value=y_true[sort_indice[:int(0.05*len(y_true))]].mean()
    return "eval_function",metric_value,True

In [20]:
def eval(y_true,y_pred):
    return np.round(eval_function(y_true,y_pred)[1]/eval_function(y_true,y_true)[1],2)

In [9]:
# 超参
params = {
'boosting_type':'gbdt',#集成方式，还包括rf,dart,goss等
'objective':'binary',
'metric':"eval_function",
'learning_rate':0.1,#学习率
'max_depth':3,#单颗树的最大深度
'num_leaves':20,#叶子节点数
'max_bins':32,#分箱数
'lambda_l1':0.1,#l1正则化权重
'lambda_l2':0.1,#l2正则化权重
'min_data_in_leaf':20,#叶子节点最小记录数
'feature_fraction':0.85,#特征抽取比例
}

trn_x = trn_df.drop(['policy_no','y'], axis=1)
trn_y = trn_df['y']

val_x = dev_df.drop(['policy_no','y'], axis=1)
val_y = dev_df['y']

trn_data = lgb.Dataset(trn_x, trn_y)
val_data = lgb.Dataset(val_x, val_y)

reg = lgb.train(params = params,
                train_set = trn_data,
                num_boost_round = 200,#最大树数量
                early_stopping_rounds = 20,#如何验证集效果在20轮中没有明显变好，就终止
                feval=eval_function,
                valid_sets = [val_data])

In [22]:
test_x = test_df.drop(['policy_no','y'], axis=1)
test_y = test_df['y']
print("训练集预测结果",eval(trn_y.values,reg.predict(trn_x)))
print("验证集预测结果",eval(val_y.values,reg.predict(val_x)))
print("测试集预测结果",eval(test_y.values,reg.predict(test_x)))

训练集预测结果 0.51
验证集预测结果 0.3
测试集预测结果 0.32


根据预测我们该接下来该如何调参数？可能会有如下策略    
1）调整树的深度   
2）调整树的数量    
3）调整叶子节点数量    
4）修改目标函数   
5）修改集成方式    
6）调整学习率     
7）添加训练数据    
8）调整正则化权重  
9）对坏样本进行归类分析  
10）添加特征工程  
.......
这些策略，我们该如何选择？可以采用如下步骤：   
1）找问题：首先，我们需要找到影响目前模型性能的主要问题；    
2）选方法：然后选择具有针对性的方法去优化，这里往往也需要权衡选择，比如某些方法可能会有较高的收益，但是会耗费很多时间，所以这里需要一个预判，哪些方法现对花费时间少，而收益更高；     
3）做实验：组织好代码结构，验证方法的收益；   
4）重复1~3，循环下去  

这部分的内容放到后面“迭代”这一部分讲解，下面聊一下借助于lgb，我们还可以做一些有助于后续模型优化的分析

### 五.模型分析   

#### 5.1 全局：特征重要度  
决策树模型通常可以帮助我们判断特征的重要度，往往有如下的几种评估方式：    
1）importance_type=split（默认值）:特征被切分过的次数；    
2）importance_type=gain:特征被切分时的loss下降值；  
3）importance_type=cover:特征被切分时的样本量(对于mse损失函数)；   

In [10]:
# importance_df=pd.DataFrame({"imp":reg.feature_importance(importance_type="split"),"col":trn_x.columns})
# importance_df.sort_values("imp",ascending=False)[:]

In [11]:
# importance_df=pd.DataFrame({"imp":reg.feature_importance(importance_type="gain"),"col":trn_x.columns})
# importance_df.sort_values("imp",ascending=False)[:]

#### 5.2 局部：贡献归因    
计算每个特征对于预测值的贡献多少，最常用的是shaply，它具有很好的性质，下面是预测的训练集的第一条数据，可以发现每个因子贡献再加上期望值就等于模型的预测值(不过要加上最后的激活函数)

In [12]:
# contrib_df=pd.DataFrame({"contrib":reg.predict(trn_x.loc[0],pred_contrib=True)[0].tolist(),"col":trn_x.columns.tolist()+['_bias']})
# contrib_df

In [26]:
reg.predict(trn_x.loc[0])

array([0.16973732])

In [27]:
1.0/(1.0+np.exp(-1*contrib_df['contrib'].sum()))

0.16973732195200125

#### 5.3 多因子分析    
决策树本质将整个样本集通过特征划分为许许多多的小区域，每一个叶子节点就代表了一个小区域，了解这些小区域内目标均值等统计量，可以帮助我们做进一步的分析或者特征工程

In [28]:
#读取权重信息
#树id,叶子节点id,叶子节点路径,叶子节点cover,叶子节点取值
import pandas as pd
class TreeLeafPath(object):
    def __init__(self,feature_names):
        self.info=[]
        self.feature_names=feature_names
        
    def CalInfo(self,total_tree):
        def G(tree,tree_index,path):
            if tree.get('leaf_value') is not None:
                # 叶子节点
                self.info.append([tree_index,tree.get('leaf_index'),path,tree.get("leaf_count"),tree.get("leaf_value")])
            else:
                # 读取路径信息
                split_feature_id=tree['split_feature']
                try:
                    threshold=np.round(float(tree['threshold']),2)
                except:
                    threshold=tree['threshold']
                if path=="":
                    G(tree['left_child'],tree_index,path+self.feature_names[split_feature_id]+"<="+str(threshold))
                    G(tree['right_child'],tree_index,path+self.feature_names[split_feature_id]+">"+str(threshold))
                else:
                    G(tree['left_child'],tree_index,path+"&"+self.feature_names[split_feature_id]+"<="+str(threshold))
                    G(tree['right_child'],tree_index,path+"&"+self.feature_names[split_feature_id]+">"+str(threshold))
                
        for item in total_tree['tree_info']:
            tree = item['tree_structure']
            G(tree,item['tree_index'],"")
        rst=pd.DataFrame(self.info,columns=["tree_index","leaf_index","path","leaf_count","leaf_value"])
        rst=rst.sort_values(['tree_index','leaf_index'])
        return rst

In [29]:
tlp=TreeLeafPath(feature_names=trn_df.columns.tolist())
tree_info=tlp.CalInfo(reg.dump_model())

In [30]:
tree_info['predict_value']=tree_info['leaf_value'].apply(lambda x:1.0/(1.0+np.exp(-1*x)))#sigmoid

In [13]:
# tree_info